# Homework with McDonald's sentiment data

## Imaginary problem statement

McDonald's receives **thousands of customer comments** on their website per day, and many of them are negative. Their corporate employees don't have time to read every single comment, but they do want to read a subset of comments that they are most interested in. In particular, the media has recently portrayed their employees as being rude, and so they want to review comments about **rude service**.

McDonald's has hired you to develop a system that ranks each comment by the **likelihood that it is referring to rude service**. They will use your system to build a "rudeness dashboard" for their corporate employees, so that employees can spend a few minutes each day examining the **most relevant recent comments**.

## Description of the data

Before hiring you, McDonald's used the [CrowdFlower platform](http://www.crowdflower.com/data-for-everyone) to pay humans to **hand-annotate** about 1500 comments with the **type of complaint**. The complaint types are listed below, with the encoding used in the data listed in parentheses:

- Bad Food (BadFood)
- Bad Neighborhood (ScaryMcDs)
- Cost (Cost)
- Dirty Location (Filthy)
- Missing Item (MissingFood)
- Problem with Order (OrderProblem)
- Rude Service (RudeService)
- Slow Service (SlowService)
- None of the above (na)

## Task 1

Read **`mcdonalds.csv`** into a pandas DataFrame and examine it. (It can be found in the **`data`** directory of the course repository.)

- The **policies_violated** column lists the type of complaint. If there is more than one type, the types are separated by newline characters.
- The **policies_violated:confidence** column lists CrowdFlower's confidence in the judgments of its human annotators for that row (higher is better).
- The **city** column is the McDonald's location.
- The **review** column is the actual text comment.

In [35]:
import pandas as pd
from datetime import datetime

In [63]:
types = dict.fromkeys(('_unit_id', '_trusted_judgements'), int)
#types['_unit_state'] = bool
#types['_last_judgment_at'] = datetime.strftime()
#print(types)
mcd = pd.read_csv('../data/mcdonalds.csv', dtype=types)
print(mcd.head(2))
mcd['city'] = mcd['city'].astype('category')
mcd['_unit_state'] = mcd['_unit_state'].astype('category')
print(mcd.memory_usage(deep = True))
#mcd.city.value_counts()
#print(mcd.columns)
print(mcd.shape)

    _unit_id _golden _unit_state  _trusted_judgments _last_judgment_at  \
0  679455653   False   finalized                   3      2/21/15 0:36   
1  679455654   False   finalized                   3      2/21/15 0:27   

                       policies_violated policies_violated:confidence  \
0  RudeService\r\nOrderProblem\r\nFilthy      1.0\r\n0.6667\r\n0.6667   
1                            RudeService                            1   

      city  policies_violated_gold  \
0  Atlanta                     NaN   
1  Atlanta                     NaN   

                                              review  Unnamed: 10  
0  I'm not a huge mcds lover, but I've been to be...          NaN  
1  Terrible customer service. ŒæI came in at 9:30...          NaN  
Index                                80
_unit_id                           6100
_golden                            1525
_unit_state                        1591
_trusted_judgments                12200
_last_judgment_at                10522

## Task 2

Remove any rows from the DataFrame in which the **policies_violated** column has a **null value**. Check the shape of the DataFrame before and after to confirm that you only removed about 50 rows.

- **Note:** Null values are also known as "missing values", and are encoded in pandas with the special value "NaN". This is distinct from the "na" encoding used by CrowdFlower to denote "None of the above". Rows that contain "na" should **not** be removed.
- **Hint:** [How do I handle missing values in pandas?](https://www.youtube.com/watch?v=fCMrO_VzeL8&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=16) explains how to do this.

In [64]:
# mcd.policies_violated.value_counts()
mcd = mcd[~mcd.policies_violated.isnull()]
print(mcd.shape)

(1471, 11)


## Task 3

Add a new column to the DataFrame called **"rude"** that is 1 if the **policies_violated** column contains the text "RudeService", and 0 if the **policies_violated** column does not contain "RudeService". The "rude" column is going to be your response variable, so check how many zeros and ones it contains.

- **Hint:** [How do I use string methods in pandas?](https://www.youtube.com/watch?v=bofaC0IckHo&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=12) shows how to search for the presence of a substring, and [How do I change the data type of a pandas Series?](https://www.youtube.com/watch?v=V0AWyzVMf54&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=13) shows how to convert the boolean results (True/False) to integers (1/0).

In [262]:
# mcd['rude'] = 
# help(mcd.policies_violated.T)
mcd['rude'] = mcd.policies_violated.str.contains('RudeService').astype(int)
mcd.rude.head(10)
mcd.rude.value_counts()

0    968
1    503
Name: policies_violated, dtype: int64

## Task 4

1. Define X (the **review** column) and y (the **rude** column).
2. Split X and y into training and testing sets (using the parameter **`random_state=1`**).
3. Use CountVectorizer (with the **default parameters**) to create document-term matrices from X_train and X_test.

In [80]:
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

X = mcd.review
y = mcd.rude

X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state = 1)
X_train.shape, X_test.shape, Y_train.shape

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [89]:
vect = CountVectorizer()
dtm = vect.fit_transform(X_train)
print(dtm.shape)
# dtm.toarray
dtm_test = vect.transform(X_test)
print(dtm_test.shape)

(1103, 7300)
(368, 7300)


## Task 5

Fit a Multinomial Naive Bayes model to the training set, calculate the **predicted probabilites** (not the class predictions) for the testing set, and then calculate the **AUC**. Repeat this task using a logistic regression model to see which of the two models achieves a better AUC.

- **Note:** Because McDonald's only cares about ranking the comments by the likelihood that they refer to rude service, **classification accuracy** is not the relevant evaluation metric. **Area Under the Curve (AUC)** is a more useful evaluation metric for this scenario, since it measures the ability of the classifier to assign higher predicted probabilities to positive instances than to negative instances.

- **Hint:** [Evaluating a classification model](https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb) explains how to calculate predicted probabilities and AUC, and my [blog post and video](http://www.dataschool.io/roc-curves-and-auc-explained/) explain AUC in-depth.

In [90]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(dtm, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [115]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, classification_report, roc_auc_score

Y_pred = nb.predict(dtm_test)
print(Y_pred[:2])
print(confusion_matrix(Y_test, Y_pred))
print('accuracy score is: ', accuracy_score(Y_test, Y_pred))

Y_pred = nb.predict_proba(dtm_test)
print(Y_pred[:2])
print('auc score is :', roc_auc_score(Y_test, Y_pred[:, 1]))

[1 0]
[[207  26]
 [ 48  87]]
accuracy score is:  0.798913043478
[[  7.65356960e-04   9.99234643e-01]
 [  7.40757029e-01   2.59242971e-01]]
auc score is : 0.842600540455


In [125]:
#Auc has better score than the binary classification for naivw bayes 

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(dtm, Y_train)
Y_pred_logreg = logreg.predict(dtm_test)
accuracy_score(Y_test, Y_pred_logreg)
roc_auc_score(Y_test, logreg.predict_proba(dtm_test)[:, 1])


0.8233667143538389

In [ ]:
#NB has the better prediction

## Task 6

Using either Naive Bayes or logistic regression (whichever one had a better AUC in the previous step), try **tuning CountVectorizer** using some of the techniques we learned in class. Check the testing set **AUC** after each change, and find the set of parameters that increases AUC the most.

- **Hint:** It is highly recommended that you adapt the **`tokenize_test()`** function from class for this purpose, since it will allow you to iterate quickly through different sets of parameters.

In [302]:
def tokenize_test(vect, y_train = Y_train, y_test = Y_test):
    #fit the input data X and Y
    dtm_train = vect.fit_transform(X_train)
    dtm_test = vect.transform(X_test)
    print('CountVectorizer is: ', vect)
    #print('Stop words given to the model are: ', vect.get_stop_words())
    print('Stop words derived from the model are: ', vect.stop_words_)
    print('dtm shape is: ', dtm_train.shape)
    
    #use naive bayes for model predictions
    nb = MultinomialNB()
    nb.fit(dtm_train, y_train)
    
    #predict now
    y_class = nb.predict(dtm_test)
    
    y_tuple = (y_test, y_class)
    print('accuracy of the model is : ', accuracy_score(*y_tuple))
    print('confusion matrix is: \n', confusion_matrix(*y_tuple))
    
    y_prob = nb.predict_proba(dtm_test)[:,1]
    print('auc score is: \n', roc_auc_score(y_test, y_prob))
    return vect

# tokenize_test(CountVectorizer())

In [155]:
#approach 1
tokenize_test(CountVectorizer())

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words given to the model are:  None
Stop words derived from the model are:  set()
dtm shape is:  (1103, 7300)
accuracy of the model is :  0.798913043478
confusion matrix is: 
 [[207  26]
 [ 48  87]]
auc score is: 
 0.842600540455


In [141]:
#Lets reduce features since we have 7300 features
tokenize_test(CountVectorizer(max_features=5000))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
dtm shape is:  (1103, 5000)
accuracy of the model is :  0.788043478261
confusion matrix is: 
 [[203  30]
 [ 48  87]]
auc score is: 
 0.840263869019


In [151]:
#Lets use df_max
tokenize_test(CountVectorizer(min_df=0, max_df=200))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=0,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
dtm shape is:  (1103, 7239)
accuracy of the model is :  0.798913043478
confusion matrix is: 
 [[203  30]
 [ 44  91]]
auc score is: 
 0.848307105389


In [153]:
#score has improved; let see if can use stop_words
tokenize_test(CountVectorizer(min_df=0, max_df=200))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=0,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words are:  {'like', 'about', 'that', 'to', 'can', 'get', 'time', 'so', 'me', 'thru', 'be', 'order', 'don', 'just', 'is', 'drive', 'no', 'and', 'out', 'food', 'have', 'they', 'here', 'if', 'not', 'or', 'what', 'are', 'up', 'been', 'when', 'mcdonalds', 'it', 'go', 'your', 'my', 'location', 'were', 'all', 'an', 'on', 'one', 'at', 'you', 'had', 'in', 'with', 'service', 'only', 'because', 'mcdonald', 'for', 'the', 'there', 'was', 'as', 'but', 'of', 'this', 'place', 'from'}
dtm shape is:  (1103, 7239)
accuracy of the model is :  0.798913043478
confusion matrix is: 
 [[203  30]
 [ 44  91]]
auc scor

In [158]:
#score has improved; let see if can use stop_words
tokenize_test(CountVectorizer(min_df=0, max_df=200, stop_words='english'))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=0,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words derived from the model are:  {'just', 'like', 'don', 'drive', 'food', 'mcdonald', 'mcdonalds', 'location', 'time', 'order', 'place', 'service'}
dtm shape is:  (1103, 7008)
accuracy of the model is :  0.796195652174
confusion matrix is: 
 [[202  31]
 [ 44  91]]
auc score is: 
 0.853012239708


In [163]:
#score has improved; let see if can use stop_words
tokenize_test(CountVectorizer(min_df=0, max_df=200, stop_words='english', lowercase=False, ngram_range=(1,1)))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=200, max_features=None, min_df=0,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words derived from the model are:  {'just', 'drive', 'like', 'don', 'The', 'food', 'location', 'McDonald', 'time', 'McDonalds', 'order', 'This', 'place', 'service'}
dtm shape is:  (1103, 8450)
accuracy of the model is :  0.815217391304
confusion matrix is: 
 [[207  26]
 [ 42  93]]
auc score is: 
 0.854029566047


In [164]:
#score has improved; let see if can use stop_words
tokenize_test(CountVectorizer(min_df=0, max_df=200, stop_words='english', lowercase=False, ngram_range=(1,2)))

CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=200, max_features=None, min_df=0,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words derived from the model are:  {'just', 'drive', 'like', 'The', 'food', 'location', 'order', 'McDonald', 'time', 'service', 'don', 'This', 'place', 'McDonalds'}
dtm shape is:  (1103, 49888)
accuracy of the model is :  0.777173913043
confusion matrix is: 
 [[215  18]
 [ 64  71]]
auc score is: 
 0.831537116516


In [173]:
#Best model has these parameters set max_df=200, stop_words='english', lowercase=False
best_model = CountVectorizer(max_df=200, stop_words='english', lowercase=False)

## Task 7 (Challenge)

The **city** column might be predictive of the response, but we are not currently using it as a feature. Let's see whether we can increase the AUC by adding it to the model:

1. Create a new DataFrame column, **review_city**, that concatenates the **review** text with the **city** text. One easy way to combine string columns in pandas is by using the [`Series.str.cat()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.cat.html) method. Make sure to use the **space character** as a separator, as well as replacing **null city values** with a reasonable string value (such as 'na').
2. Redefine X as the **review_city** column, and re-split X and y into training and testing sets.
3. When you run **`tokenize_test()`**, CountVectorizer will simply treat the city as an extra word in the review, and thus it will automatically be included in the model! Check to see whether it increased or decreased the AUC of your **best model**.

In [220]:
mcd['review_city'] = mcd.review.str.cat(mcd.city, sep=' ', na_rep='')
print(mcd.review_city[1])

X_train, X_test, Y_train, Y_test = train_test_split(mcd['review_city'], mcd.rude, random_state=1)
X_train.shape, X_test.shape

mcd.shape
tokenize_test(best_model)
X_train.isnull()
mcd.columns
mcd.shape
mcd.review_city.isnull().sum()
#Qn: Why isnull() true for unicode non latin chars?
mcd[mcd.review_city.isnull()].iloc[1].review_city

Terrible customer service. ŒæI came in at 9:30pm and stood in front of the register and no one bothered to say anything or help me for 5 minutes. ŒæThere was no one else waiting for their food inside either, just outside at the window. Œæ I left and went to Chickfila next door and was greeted before I was all the way inside. This McDonalds is also dirty, the floor was covered with dropped food. Obviously filled with surly and unhappy workers. Atlanta
CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=200, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words derived from the model are:  {'This', 'just', 'drive', 'like', 'Vegas', 'The', 'don', 'food', 'location', 'McDonald', 'time', 'McDon

'Ran in for a quick breakfast. Ordered the petite pastries and found a red hair stuck on one of my pastries. Disgusting. Beware! '

In [224]:
#Making sure that city names are added as features
set(best_model.get_feature_names()) & set(mcd.city)

{'Atlanta', 'Chicago', 'Cleveland', 'Dallas', 'Houston', 'Portland'}

## Task 8 (Challenge)

The **policies_violated:confidence** column may be useful, since it essentially represents a measurement of the training data quality. Let's see whether we can improve the AUC by only training the model using higher-quality rows!

To accomplish this, your first sub-task is to **calculate the mean confidence score for each row**, and then store those mean scores in a new column. For example, the confidence scores for the first row are `1.0\r\n0.6667\r\n0.6667`, so you should calculate a mean of `0.7778`. Here are the suggested steps:

1. Using the [`Series.str.split()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html) method, convert the **policies_violated:confidence** column into lists of one or more "confidence scores". Save the results as a new DataFrame column called **confidence_list**.
2. Define a function that calculates the mean of a list of numbers, and pass that function to the [`Series.apply()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html) method of the **confidence_list** column. That will calculate the mean confidence score for each row. Save those scores in a new DataFrame column called **confidence_mean**.
    - **Hint:** [How do I apply a function to a pandas Series or DataFrame?](https://www.youtube.com/watch?v=P_q0tkYqvSk&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=30) explains how to use the `Series.apply()` method.

In [243]:
def find_mean(s):
    if isinstance(s, str):
        l = [float(x) for x in s.split(sep="\r\n")]
        return sum(l)/len(l)
    else:
        return 0
find_mean(mcd['policies_violated:confidence'].iloc[0])

0.7778

In [292]:
mcd['confidence_mean'] = mcd['policies_violated:confidence'].apply(find_mean)
mcd['policies_violated:confidence'].head(4), mcd['confidence_mean'].head(4)

(0    1.0\r\n0.6667\r\n0.6667
 1                          1
 2                 1.0\r\n1.0
 3                     0.6667
 Name: policies_violated:confidence, dtype: object, 0    0.7778
 1    1.0000
 2    1.0000
 3    0.6667
 Name: confidence_mean, dtype: float64)

Your second sub-task is to **remove lower-quality rows from the training set**, and then repeat the model building and evaluation process. Here are the suggested steps:

1. Remove all rows from X_train and y_train that have a **confidence_mean lower than 0.75**. Check their shapes before and after to confirm that you removed about 300 rows.
2. Use the **`tokenize_test()`** function to check whether filtering the training data increased or decreased the AUC of your **best model**.
    - **Hint:** Even though X_train and y_train are separate from the mcd DataFrame, they can still be filtered using a boolean Series generated from mcd because all three objects share the same index.
    - **Note:** It's important that we don't remove any rows from the testing set (X_test and y_test), because the testing set should be representative of the real-world data we will encounter in the future (which will contain both high-quality and low-quality rows).

#I didnt see note #2
(mcd['confidence_mean'] > 0.75).sum()
X = mcd[mcd['confidence_mean'] > 0.75]['review_city']
y = mcd[mcd['confidence_mean'] > 0.75]['rude']
X.shape, y.shape
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=1)
X = mcd[mcd['confidence_mean'] > 0.75]['review_city']
y = mcd[mcd['confidence_mean'] > 0.75]['rude']
tokenize_test(best_model)
X_train.shape, X_test.shape

In [303]:
filter_series = mcd['confidence_mean'] > 0.75
X = mcd['review_city']
y = mcd['rude']
X.shape, y.shape
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=1)
X_train = X_train[filter_series]
Y_train = Y_train[filter_series]

tokenize_test(best_model)
X_train.shape, X_test.shape, Y_train.shape


CountVectorizer is:  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=200, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
Stop words derived from the model are:  {'drive', 'Vegas', 'The', 'food', 'McDonald', 'time', 'Las', 'order', 'service'}
dtm shape is:  (799, 6679)
accuracy of the model is :  0.788043478261
confusion matrix is: 
 [[199  34]
 [ 44  91]]
auc score is: 
 0.841297091082


((799,), (368,), (799,))

In [282]:
filter_series.head()
Y_train.index

Int64Index([   0,    1,    2,    4,    8,    9,   10,   11,   13,   14,
            ...
            1510, 1511, 1513, 1515, 1516, 1517, 1518, 1519, 1521, 1524],
           dtype='int64', length=1067)

## Task 9 (Challenge)

New comments have been submitted to the McDonald's website, and you need to **score them with the likelihood** that they are referring to rude service.

1. Before making predictions on out-of-sample data, it is important to re-train your model on all relevant data using the tuning parameters and preprocessing steps that produced the best AUC above.
    - In other words, X should be defined using either **all rows** or **only those rows with a confidence_mean of at least 0.75**, whichever produced a better AUC above.
    - X should refer to either the **review column** or the **review_city column**, whichever produced a better AUC above.
    - CountVectorizer should be instantiated with the **tuning parameters** that produced the best AUC above.
    - **`train_test_split()`** should not be used during this process.
2. Build a document-term matrix (from X) called **X_dtm**, and examine its shape.
3. Read the new comments stored in **`mcdonalds_new.csv`** into a DataFrame called **new_comments**, and examine it.
4. If your model uses a **review_city** column, create that column in the new_comments DataFrame. (Otherwise, skip this step.)
5. Build a document_term matrix (from the **new_comments** DataFrame) called **new_dtm**, and examine its shape.
6. Train your best model (Naive Bayes or logistic regression) using **X_dtm** and **y**.
7. Predict the "rude probability" for each comment in **new_dtm**, and store the probabilities in an object called **new_pred_prob**.
8. Print the **full text** for each new comment alongside its **"rude probability"**. (You may need to [increase the max_colwidth](https://www.youtube.com/watch?v=yiO43TQ4xvc&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=28) to see the full text.) Examine the results, and comment on how well you think the model performed!

In [320]:
#Best prediction I got was from Task 7 
#no need to use confidence_mean 
X = mcd['review_city']
X.shape
new_best_model = CountVectorizer(max_df=200, stop_words='english', lowercase=False, decode_error='ignore')
X_dtm = new_best_model.fit_transform(X)

In [323]:
new_comments = pd.read_csv('../data/mcdonalds_new.csv')
new_comments['review_city'] = new_comments['review'].str.cat(new_comments['city'], sep=' ', na_rep='')
new_comments.iloc[0].review_city
new_dtm = new_best_model.transform(new_comments['review_city'])

In [341]:
nb = MultinomialNB()
nb.fit(X_dtm, mcd['rude'])
new_comments['rude probability'] = nb.predict_proba(new_dtm)[:,1]
new_comments['rude class'] = nb.predict(new_dtm)


In [342]:
new_comments.shape
pd.options.display.max_colwidth = 1000
new_comments[['review_city', 'rude probability', 'rude class']].sort_values('rude probability', ascending= False)

,review_city,rude probability,rude class
7,My friend and I stopped in to get a late night snack and we were refused service. The store claimed to be 24 hours and the manager was standing right there doing paper work but would not help us. The cashier was only concerned with doing things for the drive thru and said that the manager said he wasn't allowed to help us. We thought it was a joke at first but when realized it wasn't we said goodbye and they just let us leave. I work in a restaurant and this is by far the worst service I have ever seen. I know it was late and maybe they didn't want to be there but it was completely ridiculous. I think the manager should be fired. Dallas,0.999055,1
0,"Went through the drive through and ordered a #10 (cripsy sweet chili chicken wrap) without fries- the lady couldn't understand that I did not want fries and charged me for them anyways. I got the wrong order- a chicken sandwich and a large fries- my boyfriend took it back inside to get the correct order. The gentleman that ordered the chicken sandwich was standing there as well and she took the bag from my bf- glanced at the insides and handed it to the man without even offering to replace. I mean with all the scares about viruses going around... ugh DISGUSTING SERVICE. Then when she gave him the correct order my wrap not only had the sweet chili sauce on it, but the nasty (just not my first choice) ranch dressing on it!!!! I mean seriously... how lazy can you get!!!! I worked at McDonalds in Texas when I was 17 for about 8 months and I guess I was spoiled with good management. This was absolutely ridiculous. I was beyond disappointed. Las Vegas",0.996180,1
8,"Friendly people but completely unable to deliver what was ordered at the drive through. Out of my last 6 orders they got it right 3 times. Incidentally, the billing was always correct - they just could not read the order and deliver. Very frustrating! Cleveland",0.927406,1
2,Ghetto lady helped me at the drive thru. Very rude and disrespectful to the co workers. Never coming back. Yuck! Los Angeles,0.793097,1
4,"I've made at least 3 visits to this particular location just because it's right next to my office building.. and all my experience have been consistently bad. There are a few helpers taking your orders throughout the drive-thru route and they are the worst. They rush you in placing an order and gets impatient once the order gets a tad bit complicated. Don't even bother changing your mind oh NO! They will glare at you and snap at you if you want to change something. I understand its FAST food, but I want my order placed right. Not going back if I can help it. Portland",0.601795,1
3,"Close to my workplace. It was well manged before. Now it's OK. The parking can be tight sometimes. Like all McDonald's, prices are getting expensive. New York",0.086283,0
5,"Why did I revisited this McDonald's again. I needed to use the restroom facilities and the women's bathroom didn't have soap, the floor was wet, the bathroom stink, and the toilets were nasty. This McDonald's is very nasty. Houston",0.054683,0
1,"Phenomenal experience. Efficient and friendly staff. Clean restrooms, good, fast service and bilingual staff. One of the best restaurants in the chain. Chicago",0.046464,0
6,"This specific McDonald's is the bar I hold all other fast food joints to now. Been working in this area for 3 years now and gone to this location many times for drive-through pickup. Service is always fast, food comes out right, and the staff is extremely warm and polite. Atlanta",0.017995,0
9,"Having visited many McDonald's over the years, I have to say that this one is the most efficient one ever! Even though it is still fast food, the service at the drive-thru is the best. They rarely make a mistake and I never see anyone parked in the drive-thru slots where they bring food out because they don't have it ready. So, if you like McDonald's fast food, it doesn't get any better than this.",0.007530,0


I have a FP in 8. All others look fine.
Any thoughts why  I got this error?
How can I find the weights for each of the features?